410921202 資工四 林芷萱 電腦視覺 0327 Practice

###### %%writefile test
## Practice (double thresholding)
1. Input images from video file WiiPlay.mp4 with the same level number as the last two digits of your student id.
2. Convert the image from BGR to HSV.
3. Use <i>cv2.createTrackbar()</i> to create two trackbars.
3. Use <i>cv2.getTrackbarPos()</i> to get the current value of each trackbar.
4. Use <i>cv2.inRange()</i> to perform double thresholding based on the Hue channel (find out pixels with Hue value between current values of two trackbars).
5. Find the best high and low threshold to detect the blue cursor.
6. Show the output images.
7. Upload your Jupyter code file (*.ipynb)

In [2]:
import cv2
import numpy as np

# 定義一個空的函數fn，用作trackbar的回調函數，即當trackbar的值更改時將被調用
def fn(x):
    pass

cap = cv2.VideoCapture('WiiPlay.mp4')
if not cap.isOpened():
    raise IOError("Cannot open the video file")

# 設定起始&結束幀
frame_seq = 14*30
stop_frame_seq = 25*30

cap.set(cv2.CAP_PROP_POS_FRAMES , frame_seq)

ret, frame = cap.read()
cur_frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

# 創建Window和trackbar
cv2.namedWindow('HSV')
cv2.createTrackbar('Low Hue', 'HSV', 100, 170, fn)
cv2.createTrackbar('High Hue', 'HSV', 150, 170, fn)

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    
    cur_frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)   
    cv2.imshow('original', cur_frame)
    
    # BGR -> HSV
    hsv_frame = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2HSV)
    
    # 獲取trackbar'Low Hue'和'High Hue'的當前值
    low_hue = cv2.getTrackbarPos('Low Hue', 'HSV')
    high_hue = cv2.getTrackbarPos('High Hue', 'HSV')
    
    # 根據Hue值的範圍對當前幀進行色彩篩選
    lower_blue = np.array([low_hue, 20, 120])
    upper_blue = np.array([high_hue, 255, 255])
    mask = cv2.inRange(hsv_frame, lower_blue, upper_blue)
#     cv2.imshow('mask', mask)
    
    # 將篩選結果可視化並轉換成明顯的顏色
    mask_colored = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    mask_colored[np.where((mask_colored == [255, 255, 255]).all(axis=2))] = [0, 0, 255]  # mask是白色的，轉成紅色
    
    # 合併原始幀和篩選結果
    combined = cv2.bitwise_or(cur_frame, mask_colored)
    cv2.imshow('HSV', combined)
    
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == stop_frame_seq:
        break
        
    c = cv2.waitKey(1)
    if c == 27:
        break
        
cap.release()
cv2.destroyAllWindows()